In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("movies_metadata.csv")

C:\Users\Noah\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
movies.drop(["belongs_to_collection","homepage","imdb_id","revenue","spoken_languages","status","tagline","original_title","video","popularity","poster_path","production_companies","production_countries"],axis=1,inplace=True)
movies.dropna()

,adult,budget,genres,id,original_language,overview,release_date,runtime,title,vote_average,vote_count
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,81.0,Toy Story,7.7,5415.0
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,1995-12-15,104.0,Jumanji,6.9,2413.0
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,1995-12-22,101.0,Grumpier Old Men,6.5,92.0
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,127.0,Waiting to Exhale,6.1,34.0
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,1995-02-10,106.0,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...
45460,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840,en,"Yet another version of the classic epic, with ...",1991-05-13,104.0,Robin Hood,5.7,26.0
45462,False,0,"[{'id': 18, 'name': 'Drama'}]",111109,tl,An artist struggles to finish his work while a...,2011-11-17,360.0,Century of Birthing,9.0,3.0
45463,False,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,en,"When one of her hits goes wrong, a professiona...",2003-08-01,90.0,Betrayal,3.8,6.0
45464,False,0,[],227506,en,"In a small town live two brothers, one a minis...",1917-10-21,87.0,Satan Triumphant,0.0,0.0


In [4]:
movies.head()

,adult,budget,genres,id,original_language,overview,release_date,runtime,title,vote_average,vote_count
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,81.0,Toy Story,7.7,5415.0
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,1995-12-15,104.0,Jumanji,6.9,2413.0
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,1995-12-22,101.0,Grumpier Old Men,6.5,92.0
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,127.0,Waiting to Exhale,6.1,34.0
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,1995-02-10,106.0,Father of the Bride Part II,5.7,173.0


In [5]:
movies = movies.replace(0, np.nan).dropna(axis=0, how='any', subset=["vote_count","overview"])

In [6]:
C= movies['vote_average'].mean()
m= movies['vote_count'].quantile(0.7)
movies = movies.copy().loc[movies['vote_count'] >= 50]

In [7]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)
movies['score'] = movies.apply(weighted_rating, axis=1)
movies = movies.sort_values('score', ascending=False)

In [8]:
import ast
def splitGenres(x):
    biglist = []
    x = ast.literal_eval(x)
    for elem in x:
        biglist.append(elem["name"])
    return biglist
movies["genres"] = movies["genres"].apply(splitGenres)


In [9]:
genres_df = pd.get_dummies(movies['genres'].apply(pd.Series).stack()).groupby(level=0).sum()
movies = pd.concat([movies, genres_df], axis=1)
movies = movies.drop(['genres'], axis=1)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
tfidf_vectorizer = TfidfVectorizer(max_features=300, stop_words='english')  # Adjust max_features as needed
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['overview'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
pickle.dump(tfidf_vectorizer.vocabulary_, open("feature.pkl", "wb"))

In [11]:
movies.reset_index(drop=True, inplace=True)
tfidf_df.reset_index(drop=True, inplace=True)
movies = pd.concat([movies, tfidf_df], axis=1)

In [12]:
movies = movies.drop(['overview'], axis=1)

In [13]:
movies['release_date'] = pd.to_datetime(movies['release_date'])
reference_date = movies['release_date'].min()
movies['days_since_release'] = (movies['release_date'] - reference_date).dt.days
movies = movies.drop(['release_date'], axis=1)
movies = movies.dropna()

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
movies['original_language'] = le.fit_transform(movies['original_language'])
movies['adult'] = movies['adult'].replace({'True': True, 'False': False}).astype(int)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
movies = movies.drop(['id','title'], axis=1)
X = movies.drop(['score'], axis=1)  # Features
y = movies['score']  # Target variable

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
normlinmse = mean_squared_error(y_test, y_pred)
normlinr2 = r2_score(y_test, y_pred)

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from math import sqrt
scorer = make_scorer(mean_squared_error, greater_is_better=False)
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring=scorer)
print("Cross-Validation Scores:", cv_scores)
average_rmse = sqrt(-cv_scores.mean())
print("Average RMSE:", average_rmse)

Cross-Validation Scores: [-0.00912329 -0.00918564 -0.00919664 -0.01021535 -0.00942363]
Average RMSE: 0.09710256394073061


In [18]:
columns_to_keep = ['adult','budget','original_language','runtime','vote_average','vote_count','Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','Foreign','History','Horror','Music','Mystery','Romance','Science Fiction','TV Movie','Thriller','War','Western']
X = movies[columns_to_keep].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
linmse = mean_squared_error(y_test, y_pred)
linr2 = r2_score(y_test, y_pred)

In [19]:
from sklearn.preprocessing import PolynomialFeatures
degree = 2
X = movies[columns_to_keep].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
poly = PolynomialFeatures(degree)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
y_pred = poly_model.predict(X_test_poly)
polymse = mean_squared_error(y_test, y_pred)
polyr2 = r2_score(y_test, y_pred)

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Evaluate the performance of the model
rfmse = mean_squared_error(y_test, y_pred)
rfr2 = r2_score(y_test, y_pred)

In [21]:
print(f'Normal linear regression Mean Squared Error: {normlinmse}')
print(f'R-squared: {normlinr2}')
print(f'No Description linear regression Mean Squared Error: {linmse}')
print(f'R-squared: {linr2}')
print(f'No Description polynomial regression Mean Squared Error: {polymse}')
print(f'R-squared: {polyr2}')
print(f'No Description Random Forest Mean Squared Error: {rfmse}')
print(f'R-squared: {rfr2}')

Normal linear regression Mean Squared Error: 0.009090754849986756
R-squared: 0.9833663897601497
No Description linear regression Mean Squared Error: 0.008909595526865759
R-squared: 0.9836978620770074
No Description polynomial regression Mean Squared Error: 0.01969278264498603
R-squared: 0.963967560839542
No Description Random Forest Mean Squared Error: 0.00038160864538818806
R-squared: 0.9993017599114387
